## 2.3.1基于python的语料库的预处理
这里所说的预处理是指，将文本分割为单词（分词），并将分割后的单词列表转化为单词 ID 列表。

In [2]:
text = 'You say goodbye and I say hello.'
"""
这里我们使用由单个句子构成的文本作为语料库。本来文本（text）应该包含成千上万个（连续的）句子，但是，考虑到简洁性，这里先对这个小的文本数据进行预处理。
"""

'\n这里我们使用由单个句子构成的文本作为语料库。本来文本（text）应该包含成千上万个（连续的）句子，但是，考虑到简洁性，这里先对这个小的文本数据进行预处理。\n'

In [3]:
text = text.lower()

In [4]:
text = text.replace('.', ' .')

In [5]:
text

'you say goodbye and i say hello .'

In [6]:
words = text.split(' ')

In [7]:
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [ ]:
"""
首先，使用 lower() 方法将所有字母转化为小写，这样可以将句子开头的单词也作为常规单词处理。然后，将空格作为分隔符，通过 split(' ') 切分句子。考虑到句子结尾处的句号（.），我们先在句号前插入一个空格（即用“ .”替换“.”），再进行分词。
"""

In [8]:
word_to_id = {}
id_to_word = {}

In [9]:
for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [10]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [11]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [12]:
id_to_word[1]

'say'

In [13]:
word_to_id['hello']

5

In [14]:
"""
将单词列表转化为单词 ID 列表
"""
import numpy as np

corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)

In [15]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [ ]:
"""
我们就完成了利用语料库的准备工作。现在，我们将上述一系列处理实现为 preprocess() 函数
"""

In [16]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [17]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

## 2.3.2 单词的分布式表示

单词的分布式表示将单词表示为固定长度的向量。这种向量的特征在于它是用密集向量表示的。密集向量的意思是，向量的各个元 素（大 多 数）是 由 非 0 实数表示的。例如，三维分布式表示是[0.21,-0.45,0.83]

## 2.3.4 共现矩阵

我们来考虑如何基于分布式假设使用向量表示单词，最直截了当的实现方法是对周围单词的数量进行计数。具体来说，在关注某个单词的情况下，对它的周围出现了多少次什么单词进行计数，然后再汇总。这里，我们将这种做法称为“基于计数的方法”，在有的文献中也称为“基于统计的方法”。

In [18]:
import sys

sys.path.append('..')

In [19]:
from common.util import preprocess

In [20]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [21]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [22]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


单词 you 的上下文仅有 say 这个单词,这也意味着可以用向量 [0, 1, 0, 0, 0, 0, 0] 表示单词 you,接着对单词 ID 为 1 的 say 进行同样的处理,单词 say 可以表示为向量 [1, 0, 1, 0, 1, 1, 0],汇总了所有单词的共现单词的表格。这个表格的各行对应相应单词的向量。因为表格呈矩阵状，所以称为共现矩阵

In [23]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
], dtype=np.int32)

In [24]:
print(C[0])  # 单词ID为0的向量

[0 1 0 0 0 0 0]


In [25]:
print(C[4])  # 单词ID为4的向量

[0 1 0 1 0 0 0]


In [26]:
print(C[word_to_id['goodbye']])  # goodbye的向量

[0 1 0 1 0 0 0]


In [ ]:
"""
下面，我们来实现一个能直接从语料库生成共现矩阵的函数。我们把这个函数称为 create_co_matrix(corpus, vocab_size,window_size=1)，其中参数 corpus 是单词 ID 列表，参数 vocab_size 是词汇个数，window_size 是窗口大小
"""

In [27]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        # 通过窗口长度确认上下文词的下标
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            # 将当前词的向量在上下午的词的位置上+1
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

In [ ]:
"""
首先，用元素为 0 的二维数组对 co_matrix 进行初始化。然后，针对语料库中的每一个单词，计算它的窗口中包含的单词。同时，检查窗口内的单词是否超出了语料库的左端和右端，之后，我们都将使用这个函数生成共现矩阵
"""

## 2.3.5 向量间的相似度

测量向量间的相似度有很多方法，其中具有代表性的方法有向量内积或欧式距离等。虽然除此之外还有很多方法，但是在测量单词的向量表示的相似度方面，余弦相似度（cosine similarity）是很常用的。

分子是向量内积，分母是各个向量的范数。范数表示向量的大小，这里计算的是 L2 范数（即向量各个元素的平方和的平方根）。

余弦相似度直观地表示了“两个向量在多大程度上指向同一方向”。两个向量完全指向相同的方向时，余弦相似度为 1；完全指向相反的方向时，余弦相似度为 −1。

In [28]:
def cos_similarity(x, y):
    nx = x / np.sqrt(np.sum(x ** 2))  # x的正规化
    ny = y / np.sqrt(np.sum(y ** 2))  # y的正规化
    return np.dot(nx, ny)

In [ ]:
"""
这里，我们假定参数 x 和 y 是 NumPy 数组。首先对向量进行正规化，然后求两个向量的内积。这里余弦相似度的实现虽然完成了，但是还有一个问题。那就是当零向量（元素全部为 0 的向量）被赋值给参数时，会出现“除数为 0”（zero division）的错误。解决此类问题的一个常用方法是，在执行除法时加上一个微小值。这里，通过参数指定一个微小值 eps（eps 是 epsilon 的缩写），并默认 eps=1e-8（= 0.000 000 01）。这样修改后的余弦相似度的实现如下所示
"""

In [29]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

In [ ]:
"""
这里我们用了 1e-8作为微小值，在这么小的值的情况下，根据浮点数的舍入误差，这个微小值会被其他值“吸收”掉。在上面的实现中，因为这个微小值会被向量的范数“吸收”掉，所以在绝大多数情况下，加上 eps不会对最终的计算结果造成影响。而当向量的范数为 0 时，这个微小值可以防止“除数为 0”的错误。
"""

In [30]:
import sys

sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity

In [31]:
text = 'You say goodbye and I say hello.'

In [32]:
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [33]:
c0 = C[word_to_id['you']]  # you的单词向量
c1 = C[word_to_id['i']]  # i的单词向量
print(cos_similarity(c0, c1))

0.7071067691154799


## 2.3.6 相似单词的排序

余弦相似度已经实现好了，使用这个函数，我们可以实现另一个便利的函数：当某个单词被作为查询词时，将与这个查询词相似的单词按降序显示出来。这里将这个函数称为 most_similar()，通过下列参数进行实现。
most_similar(query, word_to_id, id_to_word, word_matrix, top=5)

query       查询词
word_to_id  单词到单词 ID 的字典
id_to_word  单词 ID 到单词的字典
word_matrix 汇总了单词向量的矩阵，假定保存了与各行对应的单词向量
top         显示到前几位

In [34]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 取出查询词
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 计算余弦相似度
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 基于余弦相似度，按降序输出值
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        count += 1
        if count >= top:
            return

In [35]:
import sys

sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar

In [36]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [37]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
